In [1]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [2]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


In [3]:
device_name = '/CPU:0'

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [6]:
# generator function
def make_generator_network(num_hidden_layers = 1,
                          num_hidden_units = 100,
                          num_output_units = 784):
    model = tf.keras.Sequential()
    for i in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(units=num_hidden_units,
                                       use_bias = False))
        model.add(tf.keras.layers.LeakyReLU())
        
    model.add(tf.keras.layers.Dense(units = num_output_units,
                                   activation = 'tanh'))
    return model

In [7]:
# discriminator function
def make_discriminator_network(num_hidden_layers = 1,
                              num_hidden_units = 100,
                              num_output_units = 1):
    model = tf.keras.Sequential()
    for i in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(units = num_hidden_units))
        model.add(tf.keras.layers.LeakyReLU())
        model.add(tf.keras.layers.Dropout(rate = 0.5))
    model.add(tf.keras.layers.Dense(units = num_output_units,
                                   activation = None))
    return model

In [8]:
image_size = (28, 28)
z_size = 20
mode_z = 'uniform'
gen_hidden_layers = 1
gen_hidden_size = 20    #100 in the book
disc_hidden_layers = 1
disc_hidden_size = 20   #100 in the book

In [10]:
tf.random.set_seed(1)
gen_model = make_generator_network(num_hidden_layers = gen_hidden_layers,
                                  num_hidden_units = gen_hidden_size,
                                  num_output_units = np.prod(image_size))
gen_model.build(input_shape = (None, z_size))
gen_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                400       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 784)               16464     
Total params: 16,864
Trainable params: 16,864
Non-trainable params: 0
_________________________________________________________________


In [11]:
disc_model = make_discriminator_network(num_hidden_layers=disc_hidden_layers,
                                       num_hidden_units = disc_hidden_size)
disc_model.build(input_shape = (None, np.prod(image_size)))
disc_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 20)                15700     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 20)                0         
_________________________________________________________________
dropout (Dropout)            (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 15,721
Trainable params: 15,721
Non-trainable params: 0
_________________________________________________________________


In [16]:
# defining training set
mnist_bldr = tfds.builder('mnist', version = '2.0.0')
mnist_bldr.download_and_prepare()
mnist = mnist_bldr.as_dataset(shuffle_files = False)

def preprocess(ex, mode = 'uniform'):
    image = ex['image']
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.reshape(image, [-1])
    image = image * 2 - 1.0
    if mode == 'uniform':
        input_z = tf.random.uniform(shape = (z_size, ),
                                   minval = -1.0,
                                   maxval = 1.0)
    elif mode == 'normal':
        input_z = tf.random.normal(shape = (z_size,))
    return input_z, image

mnist_trainset = mnist['train']
mnist_trainset = mnist_trainset.map(preprocess)

ValueError: not enough values to unpack (expected 3, got 1)

In [20]:
!pip install tensorflow-datasets --upgrade

Traceback (most recent call last):
  File "//anaconda3/envs/ML_2020/bin/pip", line 7, in <module>
    from pip._internal.cli.main import main
  File "//anaconda3/envs/ML_2020/lib/python3.6/site-packages/pip/_internal/cli/main.py", line 8, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "//anaconda3/envs/ML_2020/lib/python3.6/site-packages/pip/_internal/cli/autocompletion.py", line 9, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "//anaconda3/envs/ML_2020/lib/python3.6/site-packages/pip/_internal/cli/main_parser.py", line 7, in <module>
    from pip._internal.cli import cmdoptions
  File "//anaconda3/envs/ML_2020/lib/python3.6/site-packages/pip/_internal/cli/cmdoptions.py", line 22, in <module>
    from pip._internal.cli.progress_bars import BAR_TYPES
  File "//anaconda3/envs/ML_2020/lib/python3.6/site-packages/pip/_internal/cli/progress_bars.py", line 9, in <module>
    from pip._internal.utils.logging import get_in